In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

# Stabilizer operators and Barnes-Wall lattices

## Dependencies

For documentation see [AbstractAlgebra.jl](https://nemocas.github.io/AbstractAlgebra.jl/dev/matrix/) and [Hecke.jl](https://www.thofma.com/Hecke.jl/dev/)


In [2]:
# using Pkg
# Pkg.add("Latexify")
# Pkg.add("Hecke")
# Pkg.add("AbstractAlgebra")
using Latexify
using Hecke
using AbstractAlgebra


Welcome to 

    _    _           _
   | |  | |         | |
   | |__| | ___  ___| | _____
   |  __  |/ _ \/ __| |/ / _ \
   | |  | |  __/ (__|   <  __/
   |_|  |_|\___|\___|_|\_\___|
    
Version 0.30.12 ... 
 ... which comes with absolutely no warranty whatsoever
(c) 2015-2024 by Claus Fieker, Tommy Hofmann and Carlo Sircana



In [3]:
# pretty printing matrices and number field elements
@latexrecipe function nf_print(x::nf_elem)
    return string(x)
end

@latexrecipe function mat_print(x::AbstractAlgebra.Generic.MatSpaceElem)
    return Array(x)
end

## Preliminaries

Define $\mathbb{Q}(i)$ and compolex conjuagtion in $\mathbb{Q}(i)$ 

In [4]:
Qi, i = cyclotomic_field(4, "i");
_, cc = Hecke.is_cm_field(Qi);
function dagger( matrix )
    ring = base_ring(matrix)
    is_cm, ring_cc = Hecke.is_cm_field(ring);
    if is_cm 
        return transpose(map(ring_cc,matrix));
    else
        @assert is_totally_real(ring);
        return transpose(matrix);
    end 
end;

### Cyclotomic number fields

Here we numerically checks that the
roots of unity $\zeta_{2^m}^j$ for $j \in [2^{m-2}]$ form an orthonormal basis with
respect to inner product $ \langle \alpha, \beta \rangle = \frac{1}{2} \mathrm{Tr}_{K/\mathbb{Q}} (\alpha\beta^\ast)$.

In [5]:
function is_cyclotomic_quadratic_form_diagonal(k)
    E, zeta = cyclotomic_field(k)
    is_cm_ok, cc_E = Hecke.is_cm_field(E)
    @assert is_cm_ok 
    d = degree(E)
    Mdd = matrix_space(AbstractAlgebra.QQ,d,d)
    integral_basis = [ zeta^j  for j in range(0,d-1) ]
    return Mdd([tr(x*cc_E(y)) for x in integral_basis, y in integral_basis ]) == Mdd(d)
end ;

In [6]:
for k in range(1,6)
    @assert is_cyclotomic_quadratic_form_diagonal(2*2^k)
    println("cyclotomic field for root of unity 2^",k," has orthonormal basis property")
end 

cyclotomic field for root of unity 2^1 has orthonormal basis property
cyclotomic field for root of unity 2^2 has orthonormal basis property
cyclotomic field for root of unity 2^3 has orthonormal basis property
cyclotomic field for root of unity 2^4 has orthonormal basis property
cyclotomic field for root of unity 2^5 has orthonormal basis property
cyclotomic field for root of unity 2^6 has orthonormal basis property


### Barnes-Wall lattices

Define a basis matrix $B$ for the one-qubit Barnes-Wall lattice over $\mathbb{Q}(i)$

In [7]:
M22 = matrix_space(Qi,2,2)
B = M22(Qi[1+i 1; 0 1])
latexify(B)

L"\begin{equation}
\left[
\begin{array}{cc}
i + 1 & 1 \\
0 & 1 \\
\end{array}
\right]
\end{equation}
"

#### Dual lattice

Basis matrix of a dual lattice $(B^\dagger)^{-1}$

In [8]:
B_dual = dagger(B^-1)
latexify(B_dual) 

L"\begin{equation}
\left[
\begin{array}{cc}
\frac{1}{2} \cdot i + \frac{1}{2} & 0 \\
\frac{-1}{2} \cdot i - \frac{1}{2} & 1 \\
\end{array}
\right]
\end{equation}
"

Check that $(B^\dagger)^{-1} = \frac{1}{1+i} B M$  for matrix $M$ with entries in $\mathbb{Z}[i]$ and determinant 1;
$(B^\dagger)^{-1}$ and $ \frac{1}{1+i} B$ are a basis matrices of the same lattice.

In [9]:
B_dual_vs_BW = (B^-1)*((1+i)*B_dual)
latexify(B_dual_vs_BW)

L"\begin{equation}
\left[
\begin{array}{cc}
i + 1 & -1 \\
 - i & i + 1 \\
\end{array}
\right]
\end{equation}
"

In [10]:
det(B_dual_vs_BW)

i

Check that $B$ and its element-wise complex conjuagate $B^\ast$ define the same lattice 

In [11]:
B_cc_vs_BW = (B^-1)*(map(cc,B))
latexify(B_cc_vs_BW) 

L"\begin{equation}
\left[
\begin{array}{cc}
 - i & 0 \\
0 & 1 \\
\end{array}
\right]
\end{equation}
"

In [12]:
det(B_cc_vs_BW)

-i

## Appendix

### Useful utilities

In [13]:
# Generator of an ideal, assuming it is a principal ideal 
function ideal_generator( principal_ideal )
    is_pr, gen = is_principal_with_data(principal_ideal)
    @assert is_pr
    return gen
end;

# Convert pseudo-matrix to matrix, assuming that the base ring is a principal ideal domain so all ideals are principal
function pseudo_matrix_to_matrix( pseudo_matrix )
    coef_ideals = Hecke.coefficient_ideals(pseudo_matrix)
    matr = Array(matrix(pseudo_matrix))
    m = matrix([ [ideal_generator(coef_ideals[k]) * x for x in matr[k,:]] for k in 1:nrows(matr) if !is_zero(matr[k,:]) ])
    return m
end;

function real_cyclotomic_field_as_subfield( root_of_unity_order::Int  ) 
    E_abs, root_of_unity = cyclotomic_field(root_of_unity_order);
    p = minpoly(2 + root_of_unity + root_of_unity^-1);
    real_subfield, _ = degree(p) == 1 ? rationals_as_number_field() : Hecke.number_field(p,string("xi_", root_of_unity_order));
    return real_subfield, E_abs, root_of_unity
end 

function cyclotomic_field_as_relative( root_of_unity_order::Int )
    real_subfield, E_abs, root_of_unity = real_cyclotomic_field_as_subfield(root_of_unity_order)
    E, rel_to_abs = relative_simple_extension(E_abs,real_subfield)
    relative_root_of_unity = inv(rel_to_abs)(root_of_unity)
    return E, inv(rel_to_abs);
end;

function as_integer_matrix( matrix, lattice_basis_matrix )
    lattice_basis = transpose(lattice_basis_matrix)
    in_lattice_basis = lattice_basis^(-1) * matrix * lattice_basis
    E, to_rel = absolute_simple_field(base_ring(matrix))
    to_abs = inv(to_rel)
    O_E = maximal_order(E)
    M = change_base_ring(O_E,map(to_abs,in_lattice_basis))
    d = nrows(M)
    reduce(vcat,[reduce(hcat,[representation_matrix(M[r,c]) for c in 1:d ]) for r in 1:d])
end;

function list_tuples(length, max_value)
    @assert length >= 0
    if length == 0 
        return [ [] ]
    else 
        prev =  list_tuples(length-1, max_value) 
        return [vcat([k],pe) for k in 0:max_value-1 for pe in prev]
    end 
end; 

### Generators of some finite groups common in Quantum Computing

#### (Complex) Clifford groups

In [14]:
function clifford_group_generators( number_of_qubits )
    F, i = cyclotomic_field(4,"i");
    s2 = 1 + i
    I = F[1 0; 0 1]
    iI = F[i 0;0 i]
    X = F[0 1;1 0]
    Z = F[1 0; 0 -1]
    H = F[1/s2 1/s2; 1/s2 -1/s2]
    S = F[1 0; 0 i]
    qubit_gens = [X,Z,H,S,dagger(S)]

    function at(matrix, qubit)
        reduce(kronecker_product,[ x==qubit ? matrix : I for x in 1:number_of_qubits ])
    end 

    function cx( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(X,target)
    end  

    I_full = at(I,1)
    center = [ i^d * I_full  for d in 0:3 ]

    if number_of_qubits == 1 
        return center, qubit_gens
    else 
        q_gens = [ at(qm,qubit) for qubit in 1:number_of_qubits for qm in qubit_gens ]
        cx_gens = [cx(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 != q2 ]
        return center, vcat(q_gens, cx_gens)
    end 
end;

#### Real Clifford groups

In [15]:
function real_clifford_group_generators( number_of_qubits )
    F, sqrt2 = quadratic_field(2);
    I = F[1 0; 0 1]
    mI = F[-1 0;0 -1]
    X = F[0 1;1 0]
    Z = F[1 0; 0 -1]
    H = F[1/sqrt2 1/sqrt2; 1/sqrt2 -1/sqrt2]
    qubit_gens = [X,Z,H]

    function at(matrix, qubit)
        reduce(kronecker_product,[ x==qubit ? matrix : I for x in 1:number_of_qubits ])
    end 

    function cx( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(X,target)
    end  

    I_full = at(I,1)
    center = [ I_full, -I_full ]

    if number_of_qubits == 1 
        return center, qubit_gens
    else 
        q_gens = [ at(qm,qubit) for qubit in 1:number_of_qubits for qm in qubit_gens ]
        cx_gens = [cx(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 != q2 ]
        return center, vcat(q_gens, cx_gens)
    end 
end;

#### Qudit Clifford groups

In [16]:
function qudit_clifford_group_generators( number_of_qudits, d = 3 )
    @assert number_of_qudits <= 2
    E, root_of_unity = cyclotomic_field(d, "ζ_$d")
    w = root_of_unity
    X = matrix([[ i - 1 == j % d ? E(1) : E(0) for i in 1:d ] for j in 1:d])
    Z = matrix([[ i == j ? w^(j-1) : E(0) for i in 1:d ] for j in 1:d])
    wI = matrix([[ i == j ? w : E(0) for i in 1:d ] for j in 1:d])
    I = matrix([[ i == j ? E(1) : E(0) for i in 1:d ] for j in 1:d])
    S = matrix([[ i == j ? w^((j-1)*(j-1+d)) : E(0) for i in 1:d ] for j in 1:d])
    sqrt_d = d % 4 == 1 ? sqrt(E(d)) : sqrt(E(-d))
    H = matrix([[ 1/sqrt_d * w^((j-1)*(i-1)) for i in 1:d ] for j in 1:d])
    qudit_gens = [X,Z,S,H]
    @assert reduce( (x,y) -> x && y, [ g*dagger(g) == parent(g)(1) for g in qudit_gens ])

    function at(matrix, qudit)
        reduce(kronecker_product,[ x==qudit ? matrix : I for x in 1:number_of_qudits ])
    end
    
    I_full = at(I,1)
    center = reduce(vcat,([ [w^j * I_full, -w^j * I_full]  for j in 0:d-1 ]))
    
    if number_of_qudits == 1 
        return center, qudit_gens
    else 
        q_gens = [ at(qm,qudit) for qudit in 1:number_of_qudits for qm in qudit_gens ]
        idx = [ (i,j) for i in 1:d for j in 1:d ]
        sum_12 = matrix([[ ir == ic && (jc - 1 == (jr-1 + ic -1) % d) ? E(1) : E(0) for (ic,jc) in idx ] for (ir,jr) in idx ])
        sum_21 = matrix([[ jr == jc && (ic - 1 == (ir-1 + jc -1) % d) ? E(1) : E(0) for (ic,jc) in idx ] for (ir,jr) in idx ])
        sum_gens = [ sum_12, sum_21 ]
        @assert reduce( (x,y) -> x && y, [ g*dagger(g) == parent(g)(1) for g in sum_gens ])
        return center, vcat(q_gens, sum_gens)
    end 
end;

#### Rational subgroup of a Real Clifford group

In [17]:
function rational_clifford_group_generators( number_of_qubits )
    F,_ = rationals_as_number_field()
    I = F[1 0; 0 1]
    mI = F[-1 0;0 -1]
    X = F[0 1;1 0]
    Z = F[1 0; 0 -1]
    H = F[1 1; 1 -1]
    qubit_gens = [X,Z]

    function at(matrix, qubit)
        reduce(kronecker_product,[ x==qubit ? matrix : I for x in 1:number_of_qubits ])
    end 

    function cx( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(X,target)
    end  

    function cz( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(Z,target)
    end 

    I_full = at(I,1)
    center = [ I_full, -I_full ]

    if number_of_qubits == 1 
        return center, qubit_gens
    else 
        q_gens = [ at(qm,qubit) for qubit in 1:number_of_qubits for qm in qubit_gens ]
        cx_gens = [cx(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 != q2 ]
        cz_gens = [cz(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 < q2 ]
        h_gens = [at(H,q1)*at(H,q2)/2 for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 < q2 ]
        return center, vcat(q_gens, cx_gens, cz_gens, h_gens)
    end 
end;

#### Dihedral groups

In [18]:
function dihedral_group_generators( number_of_qubits, root_of_unity = 8 )
    F, zeta = cyclotomic_field(root_of_unity, "ζ_$root_of_unity");
    I = F[1 0; 0 1]
    X = F[0 1;1 0]
    Z = F[1 0;0 -1]
    D = F[1 0; 0 zeta]
    qubit_gens = [X,Z,D]

    function at(matrix, qubit)
        reduce(kronecker_product,[ x==qubit ? matrix : I for x in 1:number_of_qubits ])
    end 

    function cx( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(X,target)
    end  

    I_full = at(I,1)
    if root_of_unity % 2 == 0
        center = [ zeta^d * I_full  for d in 0:root_of_unity-1 ]
    else
        center = reduce(vcat,[ [zeta^d * I_full, -zeta^d * I_full]  for d in 0:root_of_unity-1 ])
    end 

    if number_of_qubits == 1 
        return center, qubit_gens
    else 
        q_gens = [ at(qm,qubit) for qubit in 1:number_of_qubits for qm in qubit_gens ]
        cx_gens = [cx(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 != q2 ]
        return center, vcat(q_gens, cx_gens)
    end 
end;

#### Phased linear groups

In [19]:
function phased_linear_group_generators( number_of_qubits, root_of_unity = 8 )
    F, zeta = cyclotomic_field(root_of_unity, "ζ_$root_of_unity");
    I = F[1 0; 0 1]
    X = F[0 1;1 0]
    Z = F[1 0;0 -1]
    D = F[1 0; 0 zeta]
    qubit_gens = [X,Z]

    function at(matrix, qubit)
        reduce(kronecker_product,[ x==qubit ? matrix : I for x in 1:number_of_qubits ])
    end 

    function cx( control, target )
        Zc = at(Z,control);
        Ic = at(I,control);
        ((Ic+Zc)/2)*at(I,target) + ((Ic-Zc)/2)*at(X,target)
    end  

    I_full = at(I,1)
    if root_of_unity % 2 == 0
        center = [ zeta^d * I_full  for d in 0:root_of_unity-1 ]
    else
        center = reduce(vcat,[ [zeta^d * I_full, -zeta^d * I_full]  for d in 0:root_of_unity-1 ])
    end 

    diag_gens = [ diagonal_matrix(map(x-> zeta^x, tuple)) for tuple in list_tuples(2^number_of_qubits,root_of_unity) ]
    if number_of_qubits == 1 
        return center, vcat(qubit_gens,diag_gens)
    else 
        q_gens = [ at(qm,qubit) for qubit in 1:number_of_qubits for qm in qubit_gens ]
        cx_gens = [cx(q1,q2) for q1 in 1:number_of_qubits for q2 in 1:number_of_qubits if q1 != q2 ]
        return center, vcat(q_gens, cx_gens, diag_gens)
    end 
end;

### Lattice from group generators 

In [20]:
function lattice_basis_from_group_generators( center_and_generators )
    center, generators = center_and_generators
    # calculate orbit of |0> under the generators using a simple breadth first search 
    dim = ncols(generators[1])
    field = base_ring(generators[1])
    root = [ k == 1 ? field(1) : field(0) for k in 1:dim ]
    open_list = [c * root for c in center]
    result = Set(open_list)
    current_node_id = 1
    while current_node_id <= length(open_list)
        node = open_list[current_node_id]
        current_node_id = current_node_id + 1
        for g in generators
            new_node = g * node 
            if new_node in result
            else 
                push!(result,new_node)
                push!(open_list,new_node)
            end
        end 
        if current_node_id % 10000 == 0 
            print(".") # we are going to print some dots for very big orbits
        end
    end
    # make a pseudo-matrix describing a sum of orbit vector times ring of integers
    basis_matrix =  matrix(open_list)
    ring_of_integers = maximal_order(field)
    one_ideal = ideal(ring_of_integers,1)
    basis_pseudo_matrix = pseudo_matrix(basis_matrix,[one_ideal for j in 1:length(open_list)])
    # make basis compact using HNF for pseudo-matrices
    pseudo_basis_hnf = pseudo_hnf(basis_pseudo_matrix)
    return pseudo_matrix_to_matrix(pseudo_basis_hnf), length(open_list)
end;

function lattice_from_basis( basis_matrix, is_hermitian )
    field = base_ring(basis_matrix)
    if is_hermitian
        V = hermitian_space(field,nrows(basis_matrix))
        return lattice(V,basis_matrix)
    else 
        V = quadratic_space(field,nrows(basis_matrix))
        return lattice(V,basis_matrix)
    end 
end;

function number_of_minimal_vectors( L )
    abs_basis = absolute_basis(L)
    d = length(abs_basis)
    V = ambient_space(L) 
    integer_gram = matrix([[ absolute_tr(inner_product(V,abs_basis[i],abs_basis[j])) for i in 1:d] for j in 1:d ])
    LL = integer_lattice(gram = integer_gram)
    return 2*length(shortest_vectors(LL))
end;

Simple example of computing lattice for two-qubit Real Clifford group and comparing size of the orbit to the number of minimal vectors

In [21]:
basis_example, orbit_size = lattice_basis_from_group_generators(real_clifford_group_generators(2))
num_minimal = number_of_minimal_vectors(lattice_from_basis(basis_example,false))
latexify(basis_example) |> display
println("is orbit the same size as the number of minimal vectors: ", num_minimal == orbit_size)

L"\begin{equation}
\left[
\begin{array}{cccc}
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\
0 & \frac{1}{2} \cdot \sqrt{2} & 0 & \frac{1}{2} \cdot \sqrt{2} \\
0 & 0 & \frac{1}{2} \cdot \sqrt{2} & \frac{1}{2} \cdot \sqrt{2} \\
0 & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

is orbit the same size as the number of minimal vectors: true


### Check if matrix belongs to a group specified by a set of generators

In [22]:
function is_in_the_group( elements, center_and_generators ) 
    center,generators  = center_and_generators
    open_list = center
    result = Set(open_list)
    termination_set = Set([ e for e in elements if !( e in center)])
    max_num_elements = length(termination_set)
    current_node_id = 1
    num_elements_found = 0
    # simple breadth first search that terminates when all elements has been found
    while current_node_id <= length(open_list)
        node = open_list[current_node_id]
        current_node_id = current_node_id + 1
        for g in generators
            new_node = g * node
            if new_node in result
            else 
                if new_node in termination_set
                    num_elements_found = num_elements_found + 1
                    if num_elements_found == max_num_elements
                        return true
                    end
                end 
                push!(result,new_node)
                push!(open_list,new_node)
            end
        end 
        if current_node_id % 50000 == 0 
            println("found/total: ", num_elements_found, "/" ,length(elements), " total_matrices:", length(open_list)) # we are going to print intermediate information for big searches
        end
    end
    println("total group size:", length(open_list))
    return false
end;

function is_in_the_group( elements, center_and_generators, lattice_basis )
    to_int(ms) = map(x-> map(y-> Int(y),Array(as_integer_matrix(x,lattice_basis))),ms)
    return is_in_the_group(to_int(elements),(to_int(center_and_generators[1]),to_int(center_and_generators[2])))
end;

### Check if common finite groups coincide with the automorphism groups 

In [23]:
function check_lattice_properties(B_L_ext, new_center_and_gens, is_hermitian, root_order, qudit, orbit_size, check_group_completeness = true )
    L_ext = lattice_from_basis(B_L_ext,is_hermitian)
    num_minimal = number_of_minimal_vectors(L_ext)
    is_transitive = orbit_size == num_minimal
    aut_group_order = automorphism_group_order(L_ext) / length(new_center_and_gens[1])

    if check_group_completeness
        gs_ext = automorphism_group_generators(L_ext)
        group_complete = is_in_the_group(gs_ext,new_center_and_gens,B_L_ext)
        println("qudits=",qudit," root_of_unity=",root_order ," automorphism_group_size=",aut_group_order, " transitive=", is_transitive, " group_complete=", group_complete)
    else 
        println("qudits=",qudit," root_of_unity=",root_order ," automorphism_group_size=",aut_group_order, " transitive=", is_transitive)
    end 
end 

function show_base_change(B_L)
    display(latexify(transpose(B_L)))
    # uncomment for LaTeX or Julia source specifying the basis change
    # println("base change=\n", latexify(transpose(B_L)))
    # println("base change=\n", transpose(B_L))
end 

function check_hermitian_lattice_properties(generator_map, qudit_and_roots_of_unity, include_base_case = true,  check_group_completeness = true)
    for (qudit,roots_of_unity) in qudit_and_roots_of_unity
        
        center_and_gens = generator_map(qudit)
        B_L, orbit_size = lattice_basis_from_group_generators(center_and_gens)
        
        show_base_change(B_L)
        
        if include_base_case 
            check_lattice_properties(B_L, center_and_gens, true, "<base>", qudit, orbit_size, check_group_completeness )
        end 
            
        for root_order in roots_of_unity
            E_abs, w = cyclotomic_field(root_order)
            E_rel, abs_to_rel = cyclotomic_field_as_relative(root_order)
            _, sub_to_abs = is_subfield(base_ring(B_L),E_abs)
            ring_change = x -> abs_to_rel(sub_to_abs(x))
                
            gen = [ map(ring_change,g) for g in center_and_gens[2] ]
            II = parent(gen[1])(1)
            if root_order % 2 == 1 
                center = reduce(vcat, [ [abs_to_rel(w^j) *II, -abs_to_rel(w^j) *II]  for j in 0:root_order-1 ])
            else 
                center = [ abs_to_rel(w^j) * II for j in 0:root_order-1 ]
            end 
            new_center_and_gens = (center,gen)
            B_L_ext = map(ring_change,B_L)    

            check_lattice_properties(B_L_ext, new_center_and_gens, true, root_order, qudit, orbit_size  // length(center_and_gens[1]) * length(center), check_group_completeness )
        end 
    end 
end 

function check_quadratic_lattice_properties(generator_map, qudit_and_roots_of_unity, include_base_case = true,  check_group_completeness = true)
    for (qudit,roots_of_unity) in qudit_and_roots_of_unity
        center_and_gens = generator_map(qudit)
        B_L, orbit_size = lattice_basis_from_group_generators(center_and_gens)
        
        show_base_change(B_L)

        if include_base_case 
            check_lattice_properties(B_L, center_and_gens, false, "<base>", qudit, orbit_size, check_group_completeness )
        end 
        
        for root_order in roots_of_unity
            E_abs, _, _ = real_cyclotomic_field_as_subfield(root_order)
            _, sub_to_abs = is_subfield(base_ring(B_L),E_abs)
            ring_change = x -> sub_to_abs(x)
                    
            gen = [ map(ring_change,g) for g in center_and_gens[2] ]
            center = [ parent(gen[1])(1), -parent(gen[1])(1) ]
            new_center_and_gens = (center,gen)    
            B_L_ext = map(ring_change,B_L)
            
            check_lattice_properties(B_L_ext, new_center_and_gens, false, root_order, qudit, orbit_size , check_group_completeness )
        end 
    end 
end;

#### Qubit clifford group with various base rings 

In [24]:
check_hermitian_lattice_properties(clifford_group_generators, [(1,[8,12,16,20,24,28,32]),(2,[8,12,16,20])])

L"\begin{equation}
\left[
\begin{array}{cc}
\frac{1}{2} \cdot i + \frac{1}{2} & 0 \\
\frac{1}{2} \cdot i + \frac{1}{2} & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=<base> automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=8 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=12 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=16 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=20 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=24 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=28 automorphism_group_size=24 transitive=true group_complete=true
qudits=1 root_of_unity=32 automorphism_group_size=24 transitive=true group_complete=true


L"\begin{equation}
\left[
\begin{array}{cccc}
\frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & 0 & 0 \\
\frac{1}{2} & 0 & \frac{1}{2} \cdot i + \frac{1}{2} & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=2 root_of_unity=<base> automorphism_group_size=11520 transitive=true group_complete=true
found/total: 8/10 total_matrices:80041
qudits=2 root_of_unity=8 automorphism_group_size=11520 transitive=true group_complete=true
found/total: 7/9 total_matrices:98929
qudits=2 root_of_unity=12 automorphism_group_size=11520 transitive=true group_complete=true
qudits=2 root_of_unity=16 automorphism_group_size=11520 transitive=true group_complete=true
found/total: 7/9 total_matrices:119461
qudits=2 root_of_unity=20 automorphism_group_size=11520 transitive=true group_complete=true


In [25]:
check_hermitian_lattice_properties(clifford_group_generators, [(2,[24,28,32])])

L"\begin{equation}
\left[
\begin{array}{cccc}
\frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & 0 & 0 \\
\frac{1}{2} & 0 & \frac{1}{2} \cdot i + \frac{1}{2} & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & \frac{1}{2} \cdot i + \frac{1}{2} & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=2 root_of_unity=<base> automorphism_group_size=11520 transitive=true group_complete=true
found/total: 7/11 total_matrices:126101
qudits=2 root_of_unity=24 automorphism_group_size=11520 transitive=true group_complete=true
found/total: 8/11 total_matrices:134176
found/total: 9/11 total_matrices:209600
qudits=2 root_of_unity=28 automorphism_group_size=11520 transitive=true group_complete=true
found/total: 6/10 total_matrices:140033
qudits=2 root_of_unity=32 automorphism_group_size=11520 transitive=true group_complete=true


In [26]:
check_hermitian_lattice_properties(clifford_group_generators, [(3,[])],true,false)

"\\begin{equation}\n\\left[\n\\begin{array}{cccccccc}\n\\frac{1}{4} \\cdot i + \\frac{1}{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} \\cdot i + \\frac{1}{4} & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} \\cdot i + \\frac{1}{4} & 0 & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}" ⋯ 374 bytes ⋯ " & \\frac{1}{2} \\cdot i + \\frac{1}{2} & 0 \\\\\n\\frac{1}{4} \\cdot i + \\frac{1}{4} & \\frac{1}{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot i + \\frac{1}{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot i + \\frac{1}{2} & \\frac{1}{2} \\cdot i + \\frac{1}{2} & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=3 root_of_unity=<base> automorphism_group_size=92897280 transitive=true


#### Qutrit Clifford group with various base rings 

In [27]:
check_hermitian_lattice_properties(qudit_clifford_group_generators, [(1,[9,12,15,18,21,24,27])])

L"\begin{equation}
\left[
\begin{array}{ccc}
\frac{-1}{3} \cdot \zeta_{3} + \frac{1}{3} & 0 & 0 \\
\frac{-1}{3} \cdot \zeta_{3} + \frac{1}{3} & 1 & 0 \\
\frac{-1}{3} \cdot \zeta_{3} + \frac{1}{3} & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=<base> automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=9 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=12 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=15 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=18 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=21 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=24 automorphism_group_size=216 transitive=true group_complete=true
qudits=1 root_of_unity=27 automorphism_group_size=216 transitive=true group_complete=true


In [28]:
check_hermitian_lattice_properties(qudit_clifford_group_generators, [(2,[])], true, false)

"\\begin{equation}\n\\left[\n\\begin{array}{ccccccccc}\n\\frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{3} & \\frac{2}{3} \\cdot \\zeta_{3} + \\frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{3} & 0 & \\frac{1}{3} \\cdot \\zeta_{3} + \\frac{2}{3} & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\" ⋯ 686 bytes ⋯ "& \\frac{4}{3} \\cdot \\zeta_{3} + \\frac{2}{3} & \\frac{1}{3} \\cdot \\zeta_{3} + \\frac{2}{3} & \\frac{2}{3} \\cdot \\zeta_{3} + \\frac{4}{3} & \\frac{1}{3} \\cdot \\zeta_{3} + \\frac{2}{3} & 0 & \\frac{1}{3} \\cdot \\zeta_{3} + \\frac{2}{3} & 0 & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=2 root_of_unity=<base> automorphism_group_size=4199040 transitive=true


#### Real Clifford group with various base rings

In [29]:
check_quadratic_lattice_properties(real_clifford_group_generators, [(1,[8,16,24,32]),(2,[8,16,24,32])])

L"\begin{equation}
\left[
\begin{array}{cc}
\frac{1}{2} \cdot \sqrt{2} & 0 \\
\frac{1}{2} \cdot \sqrt{2} & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=<base> automorphism_group_size=8 transitive=true group_complete=true
qudits=1 root_of_unity=8 automorphism_group_size=8 transitive=true group_complete=true
qudits=1 root_of_unity=16 automorphism_group_size=8 transitive=true group_complete=true
qudits=1 root_of_unity=24 automorphism_group_size=8 transitive=true group_complete=true
qudits=1 root_of_unity=32 automorphism_group_size=8 transitive=true group_complete=true


L"\begin{equation}
\left[
\begin{array}{cccc}
\frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot \sqrt{2} & 0 & 0 \\
\frac{1}{2} & 0 & \frac{1}{2} \cdot \sqrt{2} & 0 \\
\frac{1}{2} & \frac{1}{2} \cdot \sqrt{2} & \frac{1}{2} \cdot \sqrt{2} & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=2 root_of_unity=<base> automorphism_group_size=1152 transitive=true group_complete=true
qudits=2 root_of_unity=8 automorphism_group_size=1152 transitive=true group_complete=true
qudits=2 root_of_unity=16 automorphism_group_size=1152 transitive=true group_complete=true
qudits=2 root_of_unity=24 automorphism_group_size=1152 transitive=true group_complete=true
qudits=2 root_of_unity=32 automorphism_group_size=1152 transitive=true group_complete=true


In [30]:
check_quadratic_lattice_properties(real_clifford_group_generators, [(3,[]),(4,[])],true,false)

"\\begin{equation}\n\\left[\n\\begin{array}{cccccccc}\n\\frac{1}{4} \\cdot \\sqrt{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} \\cdot \\sqrt{2} & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} \\cdot \\sqrt{2} & 0 & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} \\cdot \\sqrt{2} & " ⋯ 276 bytes ⋯ "& \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & \\frac{1}{2} \\cdot \\sqrt{2} & 0 \\\\\n\\frac{1}{4} \\cdot \\sqrt{2} & \\frac{1}{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot \\sqrt{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot \\sqrt{2} & \\frac{1}{2} \\cdot \\sqrt{2} & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=3 root_of_unity=<base> automorphism_group_size=2580480 transitive=true


"\\begin{equation}\n\\left[\n\\begin{array}{cccccccccccccccc}\n\\frac{1}{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} & \\frac{1}{4} \\cdot \\sqrt{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} & 0 & \\frac{1}{4} \\cdot \\sq" ⋯ 1991 bytes ⋯ "dot \\sqrt{2} & \\frac{1}{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot \\sqrt{2} & \\frac{1}{4} \\cdot \\sqrt{2} & \\frac{1}{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot \\sqrt{2} & \\frac{1}{2} & \\frac{1}{2} \\cdot \\sqrt{2} & \\frac{1}{2} \\cdot \\sqrt{2} & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=4 root_of_unity=<base> automorphism_group_size=89181388800 transitive=true


#### Rational subroup of a Real Clifford group 

In [31]:
check_quadratic_lattice_properties(rational_clifford_group_generators, [(1,[8,16,24,32]),(2,[8,16,24,32])])

L"\begin{equation}
\left[
\begin{array}{cc}
1 & 0 \\
0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=<base> automorphism_group_size=4 transitive=true group_complete=true
qudits=1 root_of_unity=8 automorphism_group_size=4 transitive=true group_complete=true
qudits=1 root_of_unity=16 automorphism_group_size=4 transitive=true group_complete=true
qudits=1 root_of_unity=24 automorphism_group_size=4 transitive=true group_complete=true
qudits=1 root_of_unity=32 automorphism_group_size=4 transitive=true group_complete=true


L"\begin{equation}
\left[
\begin{array}{cccc}
\frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & 1 & 0 & 0 \\
\frac{1}{2} & 0 & 1 & 0 \\
\frac{1}{2} & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=2 root_of_unity=<base> automorphism_group_size=576 transitive=true group_complete=true
qudits=2 root_of_unity=8 automorphism_group_size=576 transitive=true group_complete=true
qudits=2 root_of_unity=16 automorphism_group_size=576 transitive=true group_complete=true
qudits=2 root_of_unity=24 automorphism_group_size=576 transitive=true group_complete=true
qudits=2 root_of_unity=32 automorphism_group_size=576 transitive=true group_complete=true


In [32]:
check_quadratic_lattice_properties(rational_clifford_group_generators, [(3,[]),(4,[])],true,false)

L"\begin{equation}
\left[
\begin{array}{cccccccc}
\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & \frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{1}{2} & 0 & 0 & 0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & \frac{1}{2} & 0 & 0 & \frac{1}{2} & 1 & 0 & 0 \\
\frac{1}{2} & 0 & \frac{1}{2} & 0 & \frac{1}{2} & 0 & 1 & 0 \\
0 & \frac{1}{2} & \frac{1}{2} & 0 & \frac{1}{2} & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=3 root_of_unity=<base> automorphism_group_size=348364800 transitive=true


"\\begin{equation}\n\\left[\n\\begin{array}{cccccccccccccccc}\n\\frac{1}{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{4} & 0 & \\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 " ⋯ 1151 bytes ⋯ " \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & 1 & 0 \\\\\n\\frac{1}{4} & 0 & 0 & \\frac{1}{2} & 0 & \\frac{1}{2} & \\frac{1}{2} & 0 & 0 & \\frac{1}{2} & \\frac{1}{2} & 0 & \\frac{1}{2} & 0 & 0 & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=4 root_of_unity=<base> automorphism_group_size=44590694400 transitive=true


In [33]:
89181388800  // 2

44590694400//1

#### Qudit Clifford group for d=5

In [34]:
check_hermitian_lattice_properties(x -> qudit_clifford_group_generators(x,5),[(1,[5])],false)

"\\begin{equation}\n\\left[\n\\begin{array}{ccccc}\n\\frac{1}{5} \\cdot \\zeta_{5}^{3} + \\frac{2}{5} \\cdot \\zeta_{5} + \\frac{2}{5} & 0 & 0 & 0 & 0 \\\\\n\\frac{1}{5} \\cdot \\zeta_{5}^{3} + \\frac{2}{5} \\cdot \\zeta_{5} + \\frac{2}{5} & \\frac{-3}{5} \\cdot \\zeta_{5}^{3} - \\frac{1}{5} \\cdot " ⋯ 668 bytes ⋯ "\\frac{-6}{5} \\cdot \\zeta_{5}^{3} - \\frac{2}{5} \\cdot \\zeta_{5}^{2} + \\frac{2}{5} \\cdot \\zeta_{5} - \\frac{4}{5} & \\frac{-3}{5} \\cdot \\zeta_{5}^{3} - \\frac{1}{5} \\cdot \\zeta_{5}^{2} + \\frac{1}{5} \\cdot \\zeta_{5} - \\frac{2}{5} & 0 & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

qudits=1 root_of_unity=5 automorphism_group_size=3000 transitive=true group_complete=true


#### Dihedral groups

In [35]:
check_hermitian_lattice_properties(x -> dihedral_group_generators(x,8),[(1,[8]),(2,[8])],false)

L"\begin{equation}
\left[
\begin{array}{cc}
1 & 0 \\
0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=8 automorphism_group_size=16 transitive=true group_complete=true


L"\begin{equation}
\left[
\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

total group size:49152
qudits=2 root_of_unity=8 automorphism_group_size=12288 transitive=true group_complete=false


#### Phased linear groups

In [36]:
check_hermitian_lattice_properties(x -> phased_linear_group_generators(x,4),[(1,[4]),(2,[4])],false)

L"\begin{equation}
\left[
\begin{array}{cc}
1 & 0 \\
0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=1 root_of_unity=4 automorphism_group_size=8 transitive=true group_complete=true


L"\begin{equation}
\left[
\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
"

qudits=2 root_of_unity=4 automorphism_group_size=1536 transitive=true group_complete=true


### Sizes of various symplectic groups

In [37]:
function symplectic_group_size( num_qudits, dimension )
    q = dimension
    n = num_qudits
    return reduce((x,y) -> x*y, [(BigInt(q^(2*i))-1) for i in 1:n])*(q^(n^2))
end 

function clifford_group_size( num_qudits, dimension )
    q = dimension
    n = num_qudits
    return symplectic_group_size(num_qudits, dimension)* q^(2*n)
end;

In [38]:
size_table =  reduce( hcat, [ [q,n,clifford_group_size(n,q)] for q in [2,3,5] for n in 1:4 ]) |> transpose 

size_table |> latexify |> display
# size_table |> latexify |> print 

L"\begin{equation}
\left[
\begin{array}{ccc}
2 & 1 & 24 \\
2 & 2 & 11520 \\
2 & 3 & 92897280 \\
2 & 4 & 12128668876800 \\
3 & 1 & 216 \\
3 & 2 & 4199040 \\
3 & 3 & 6685442749440 \\
3 & 4 & 863227576820212531200 \\
5 & 1 & 3000 \\
5 & 2 & 5850000000 \\
5 & 3 & 7140656250000000000 \\
5 & 4 & 5447874427734375000000000000000 \\
\end{array}
\right]
\end{equation}
"

### Tensor product structure of Basis matrices and modularity

In [39]:
function is_modular(M)
    A = dagger(M)*M
    F = base_ring(M)
    O_F = maximal_order(F)
    det_primes = [ ideal_generator(x[1]) for x in Hecke.factor(det(A)*O_F)]
    if length(det_primes) > 1 
        println("Not modular, it is necessary that determinant is a power of a prime of O_F")
        return false
    elseif length(det_primes) == 1 
        xi = det_primes[1];
        p = xi * O_F;
        power = valuation(det(A),p);
        if power % nrows(M) != 0 
            println("Not modular, it is necessary that determinant is a power of a prime of O_F divisible by ",nrows(M))
            denominator_power = -minimum([ valuation(x,p) for x in Array(A)[:] if x != 0]);
            @show denominator_power
            A_scaled =  xi^denominator_power * A
            A_scaled |> latexify |> display
            @show det(A_scaled) , valuation(det(A_scaled),p)
            return false
        else
            scalar_power = div(power,nrows(M))
            A_scaled = xi^(-scalar_power) * A
            return [a in O_F for a in Array(A_scaled)[:] ] |> all
        end 
    elseif length(det_primes) == 0
        return true
    end 
    return true
end;

function is_equivalent_basis(A,B)
    rel = A^-1*B;
    E = base_ring(A);
    O_E = maximal_order(E);
    @assert abs(absolute_norm(det(rel))) == 1;
    reduce( (a,b) -> a && b, Array(map( x -> E(x) in O_E, rel))[:])
end;

function is_equivalent_basis_rational(A,B)
    rel = A^-1*B;
    E = base_ring(A);
    O_E = maximal_order(E);
    @assert abs(absolute_norm(det(rel))) == 1;
    reduce( (a,b) -> a && b, Array(map( x -> isinteger(x), rel))[:])
end;

#### Qutrits 

In [40]:
B3,_ = lattice_basis_from_group_generators(qudit_clifford_group_generators(1))
B9,_ = lattice_basis_from_group_generators(qudit_clifford_group_generators(2))
rel = B9^-1 * kronecker_product(B3,B3)
display(latexify(rel))
@show absolute_norm(det(rel))

@show [is_modular(transpose(basis)) for basis in [B3,B9]];

"\\begin{equation}\n\\left[\n\\begin{array}{ccccccccc}\n - \\zeta_{3} & 1 & 0 & 0 & 2 \\cdot \\zeta_{3} & \\zeta_{3} + 5 & 0 & \\zeta_{3} + 6 & \\zeta_{3} + 6 \\\\\n0 &  - \\zeta_{3} - 1 & 0 & 0 &  - \\zeta_{3} - 1 & -1 & 0 &  - \\zeta_{3} - 3 & -2 \\\\\n0 & 0 &  - \\zeta_{3} & 0 & 0 &  - \\zet" ⋯ 125 bytes ⋯ " & 0 &  - \\zeta_{3} + 1 & -2 & 0 & -2 & -1 \\\\\n0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 0 & 0 & 0 & 0 &  - \\zeta_{3} &  - \\zeta_{3} - 1 &  - \\zeta_{3} - 1 \\\\\n0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

absolute_norm(det(rel)) = 3
Not modular, it is necessary that determinant is a power of a prime of O_F divisible by 3
denominator_power = 1


L"\begin{equation}
\left[
\begin{array}{ccc}
 - \zeta_{3} + 1 & 1 & 1 \\
 - \zeta_{3} &  - \zeta_{3} + 1 & 0 \\
 - \zeta_{3} & 0 &  - \zeta_{3} + 1 \\
\end{array}
\right]
\end{equation}
"

(det(A_scaled), valuation(det(A_scaled), p)) = (-2*ζ_3 - 1, 1)
Not modular, it is necessary that determinant is a power of a prime of O_F divisible by 9
denominator_power = 2


"\\begin{equation}\n\\left[\n\\begin{array}{ccccccccc}\n-3 \\cdot \\zeta_{3} & 2 \\cdot \\zeta_{3} + 4 &  - \\zeta_{3} + 1 & -2 \\cdot \\zeta_{3} + 2 & -2 \\cdot \\zeta_{3} + 2 &  - \\zeta_{3} &  - \\zeta_{3} + 1 &  - \\zeta_{3} &  - \\zeta_{3} \\\\\n-2 \\cdot \\zeta_{3} - 4 & -10 \\cdot \\zeta_{" ⋯ 1065 bytes ⋯ "{3} + 1 & -2 \\cdot \\zeta_{3} + 2 & 0 &  - \\zeta_{3} + 1 & -3 \\cdot \\zeta_{3} & 0 \\\\\n - \\zeta_{3} & 2 \\cdot \\zeta_{3} + 4 &  - \\zeta_{3} + 1 & -2 \\cdot \\zeta_{3} + 2 &  - \\zeta_{3} + 1 & 0 &  - \\zeta_{3} + 1 & 0 & -3 \\cdot \\zeta_{3} \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"

(det(A_scaled), valuation(det(A_scaled), p)) = (-9, 4)
[is_modular(transpose(basis)) for basis = [B3, B9]] = Bool[0, 0]


#### Real Clifford group

In [41]:
B2,_ = lattice_basis_from_group_generators(real_clifford_group_generators(1))
B4,_ = lattice_basis_from_group_generators(real_clifford_group_generators(2))
B8,_ = lattice_basis_from_group_generators(real_clifford_group_generators(3))
B16,_ = lattice_basis_from_group_generators(real_clifford_group_generators(4))

@show is_equivalent_basis(B4, kronecker_product(B2,B2))
@show is_equivalent_basis(B8, kronecker_product(B4,B2))
@show is_equivalent_basis(B16, kronecker_product(B4,B4))

@show [is_modular(transpose(basis)) for basis in [B2,B4,B8,B16]];

is_equivalent_basis(B4, kronecker_product(B2, B2)) = true
is_equivalent_basis(B8, kronecker_product(B4, B2)) = true
is_equivalent_basis(B16, kronecker_product(B4, B4)) = true
[is_modular(transpose(basis)) for basis = [B2, B4, B8, B16]] = Bool[1, 1, 1, 1]


#### Complex Clifford group 

In [42]:
BC2,_ = lattice_basis_from_group_generators(clifford_group_generators(1))
BC4,_ = lattice_basis_from_group_generators(clifford_group_generators(2))
BC8,_ = lattice_basis_from_group_generators(clifford_group_generators(3))
BC16,_ = lattice_basis_from_group_generators(clifford_group_generators(4))

@show is_equivalent_basis(BC4, kronecker_product(BC2,BC2))
@show is_equivalent_basis(BC8, kronecker_product(BC4,BC2))

@show [is_modular(transpose(basis)) for basis in [BC2,BC4,BC8]];

..............is_equivalent_basis(BC4, kronecker_product(BC2, BC2)) = true
is_equivalent_basis(BC8, kronecker_product(BC4, BC2)) = true
[is_modular(transpose(basis)) for basis = [BC2, BC4, BC8]] = Bool[1, 1, 1]


#### Rational subgroup of a Real Clifford group

In [43]:
function check_rational_basis(num_qubits = 3)
    E, i = cyclotomic_field(4);
    _, cc = Hecke.is_cm_field(E);
    qubit_BW_basis_Qi =  E[1 1; 1+i 0];
    BW_basis_Qi = num_qubits-1 == 1 ? qubit_BW_basis_Qi : reduce(kronecker_product, [qubit_BW_basis_Qi  for k in 1:num_qubits-1])
    reBasis = map(x -> Hecke.QQ(cc(x) + x)//2, BW_basis_Qi )
    imBasis = map(x -> Hecke.QQ((cc(x) - x) * (-i) )//2, BW_basis_Qi )
    basis_Q = vcat(hcat(reBasis,imBasis),hcat(-imBasis,reBasis))/2
    basis_QC = map( x -> Hecke.QQ(x), lattice_basis_from_group_generators(rational_clifford_group_generators(num_qubits))[1])
    @show det(integer_lattice(basis_Q)), det(integer_lattice(basis_QC))
    @show is_equivalent_basis_rational(basis_Q, basis_QC)
    @show is_isometric(integer_lattice(basis_Q), integer_lattice(basis_QC))
end

check_rational_basis()

(det(integer_lattice(basis_Q)), det(integer_lattice(basis_QC))) = (1//256, 1//256)
is_equivalent_basis_rational(basis_Q, basis_QC) = false
is_isometric(integer_lattice(basis_Q), integer_lattice(basis_QC)) = true


true